In [201]:
import cv2 
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [202]:
def findK(percentClarity, arr):
    k = 0
    s = 0
    arrSum = sum(arr)
    for i in arr:
        s += i
        p = (s/arrSum)*100
        k+=1
        if round(p) >= percentClarity:
            return k

def compress_channel(img_array, channel, percentage):
    specific_channel = img_array[:,:,channel]
    U, S, Vt = np.linalg.svd(specific_channel)
    k = findK(percentage, S)
    reconstructed_channel = U[:,:k] @ np.diag(S[:k]) @ Vt[:k,:]
    return reconstructed_channel

def compressed_image(img_array, percentage):
    l, w, h = img_array.shape
    newimg_array = np.zeros((l,w,h))
    for channel in range(h):
        newimg_array[:,:,channel] = compress_channel(img_array, channel, percentage)
    # i = Image.fromarray(newimg_array.astype('uint8'))
    newimg_array[:, :, [0, 2]] = newimg_array[:, :, [2, 0]]
    return newimg_array

In [203]:
# Function to extract frames

def extract_frames(vid_path):

    videoName = vid_path.split("/")[-1].split(".mp4")[0]

    directory = "extracted_frames/" + videoName
    if not os.path.exists(directory):
        os.mkdir(directory)


    # Path to video file
    videoObj = cv2.VideoCapture(vid_path)

    # checks whether frames were extracted
    success = 1
    count = 1
    while success:
        success, image = videoObj.read()
        if image is not None:
            cv2.imwrite(directory + "/" + str(count) + ".jpg", image)
            count+=1
    return videoName

In [204]:
def createVideo(path, name):

    # Define the frame rate and video size
    firstImg = path + '/' + sorted(os.listdir(path))[0]
    fps = cv2.VideoCapture(firstImg).get(cv2.CAP_PROP_FPS)
    size = (100, 100)
    count = 0
    # Define the codec to use for the video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Define the output file name and path
    output_path = 'output_videos/' + name + '.mp4'

    # Create a new video writer object
    out = cv2.VideoWriter(output_path, fourcc, fps, size)

    # Loop through the images in the folder and add them to the video
    for filename in tqdm(sorted(os.listdir(path))):
        if filename.endswith('.jpg'):
            
            img_path = os.path.join(path, filename)
            img = cv2.imread(img_path)
            i = compressed_image(img, 100)
            i[:, :, [0, 2]] = i[:, :, [2, 0]]
            cv2.imwrite(img_path, i)
            compressedImage = cv2.imread(img_path)
            out.write(compressedImage)

    # Release the video writer object and close the video file
    out.release()

In [205]:
directory = r"C:/Harrithha/IIT_Dh/College_Academics/Sem_6/MDS/MDS_2023/Assignment_1/videos/"

for video in os.listdir(directory):
    videoPath  = directory + video
    vidName = extract_frames(videoPath)
    createVideo("extracted_frames/" + vidName, vidName)

  2%|▏         | 26/1655 [00:24<25:36,  1.06it/s]


KeyboardInterrupt: 